# Data

Loading and appending the Chicago Crime dataset

In [1]:
import pandas as pd
import duckdb
import time 
import os
import requests

Set the path of the CSV files:

In [3]:
path = "./data/titanic"

files = [x for x in os.listdir(path = path) if ".csv" in x]

print(files)

['titanic.csv']


In [4]:
data = pd.concat((pd.read_csv(path +"/" + f) for f in files), ignore_index=True)
data.columns = [c.strip().replace(" ", "_").lower() for c in data.columns]
data.head()

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Source: https://platform.openai.com/examples/default-sql-translate

In [5]:
# Make sure to end the query with with a semicolon.
# Only output the pure SQL code. Do not give an explanation for your answer.

In [8]:
setup_prompt_template = """
I want you to be a SQL expert! 
Given the following SQL table \n

CREATE TABLE {} ({}) \n

Give only code, no explanantion, and ending the query with a semicolon
"""

prompt_template = """
Can you write a SQL query that returns - {}
"""
#, your job is to write queries given a user’s request, giving only code, no explanantion, and ending the query with a semicolon.

def sql_setup_prompt_generator(table_name, col_names):
    prompt = setup_prompt_template.format(table_name, col_names)
    return prompt

def sql_prompt_generator( query):
    prompt = prompt_template.format( query)
    return prompt


In [9]:
table = "data"
col_names = str(list(data.columns)).replace('[', '').replace(']', '')

p = sql_setup_prompt_generator(table_name = table, col_names = col_names)

print(p)




I want you to be a SQL expert! 
Given the following SQL table 


CREATE TABLE data ('passengerid', 'survived', 'pclass', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket', 'fare', 'cabin', 'embarked') 


Give only code, no explanantion, and ending the query with a semicolon



## Bard API Integration

In [10]:
from bardapi import Bard

token = os.environ.get("BARD_API_KEY")

In [11]:
session = requests.Session()
session.headers = {
            "Host": "bard.google.com",
            "X-Same-Domain": "1",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
            "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8",
            "Origin": "https://bard.google.com",
            "Referer": "https://bard.google.com/",
        }
session.cookies.set("__Secure-1PSID", os.getenv("BARD_API_KEY")) 
session.cookies.set("__Secure-1PSID", token) 

bard = Bard(token=token, session=session, timeout=30)

### Prompt setup

In [12]:
bard.get_answer(p)['content']

' /* Ready for your SQL queries! */ ;\n'

### Prompt follow ups

In [13]:
q1 = "How many passengers are there?"
qp = sql_prompt_generator(q1)
answer = bard.get_answer(qp)

In [14]:
answer['content']

response_text = answer['content']
startidtoken = "```sql"
endidtoken = ";"
sqlkey_select_loc = response_text.find(startidtoken)+len(startidtoken)
sqlkey_semicolon_loc = response_text.find(endidtoken)
output_query = response_text[sqlkey_select_loc:sqlkey_semicolon_loc]

print("Executing query : ")
print(output_query)

Executing query : 
ldn't complete your request. Try submitting your prompt again


In [17]:
def lang2sql(bard, query):
    """
    Takes in a prompt and a query question and returns the corresponding dataframe
    """
    try:
        qp = sql_prompt_generator(query)
        answer = bard.get_answer(qp)

        response_text = answer['content']
        startidtoken = "```sql"
        endidtoken = ";"
        sqlkey_select_loc = response_text.find(startidtoken)+len(startidtoken)
        sqlkey_semicolon_loc = response_text.find(endidtoken)
        query = response_text[sqlkey_select_loc:sqlkey_semicolon_loc]

        print("Executing query : ")
        print(query)

        query = query.replace("`", "\"")

        output_df = duckdb.sql(query).df()

        return output_df
    except Exception as e:
        print(e)
        return answer


In [18]:
query = "How many passengers survived?"

output_df = lang2sql(bard=bard, query=query)

display(output_df)

Executing query : 

SELECT SUM(survived) AS total_survived
FROM data


,total_survived
0,342.0


In [44]:
query = "How many passengers did not survive?"

output_df = lang2sql(bard=bard, query=query)

display(output_df)

Executing query : 

SELECT COUNT(*) AS num_not_survived FROM data WHERE survived = 0


,num_not_survived
0,549


In [45]:
query = "What is the most common last name of a passenger?"

output_df = lang2sql(bard=bard, query=query)

display(output_df)

Executing query : 

WITH last_names_counts AS (
  SELECT SUBSTRING_INDEX(name, ',', -1) AS last_name, COUNT(*) AS count
  FROM data
  GROUP BY last_name
)
SELECT last_name
FROM last_names_counts
ORDER BY count DESC
LIMIT 1
Catalog Error: Scalar Function with name substring_index does not exist!
Did you mean "substring"?


""


In [19]:
query = "What was the number of survivors per age group?"

output_df = lang2sql(bard=bard, query=query)

display(output_df)

Executing query : 

SELECT
    CASE WHEN age <= 15 THEN '0-15'
         WHEN age <= 30 THEN '16-30'
         WHEN age <= 45 THEN '31-45'
         WHEN age <= 60 THEN '46-60'
         ELSE '61+' END AS age_group,
    COUNT(*) AS survivors
FROM data
WHERE survived = 1
GROUP BY age_group


,age_group,survivors
0,0-15,49
1,16-30,117
2,61+,57
3,46-60,33
4,31-45,86


## Veggie buddie 

### Prompt setup


In [20]:
setup = """You are an expeert chef, who is globally aware, and caters to an international clientele with diverse culinary backgrounds. 
This global perspective is reflected in the suggestions and adaptations of recipes, considering regional tastes, available ingredients, and cultural cooking methods. 
Whether the user is from Western countries like the USA and UK, or from other parts of the world, you provide relevant vegan alternatives and cooking advice. 
This inclusive approach ensures that the advice is not only vegan-friendly but also culturally sensitive and adaptable to different regional cuisines and preferences."""


In [22]:
session = requests.Session()
session.headers = {
            "Host": "bard.google.com",
            "X-Same-Domain": "1",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
            "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8",
            "Origin": "https://bard.google.com",
            "Referer": "https://bard.google.com/",
        }
session.cookies.set("__Secure-1PSID", os.getenv("BARD_API_KEY")) 
session.cookies.set("__Secure-1PSID", token) 

bard_chef = Bard(token=token, session=session, timeout=30)

### Prompt setup

In [23]:
setup_answer = bard_chef.get_answer(setup)

In [28]:
print(
    "\n".join(setup_answer['content'].split("."))
)

Bonjour and bonjourina! Welcome to my global kitchen, where culinary borders disappear and taste buds travel the world! I'm your friendly vegan chef, here to whip up plant-powered magic for any palate, anywhere
 Tell me, where are you hailing from today? Whether you crave the hearty stews of England or the fiery curries of Thailand, I've got vegan twists that'll tantalize your taste buds without compromising tradition


Now, don't let the "vegan" label scare you off
 Think of it as an open invitation to explore a whole new spectrum of flavors and textures! We'll swap creamy cheese for nutty tahini, replace meat with protein-packed lentils, and turn ordinary veggies into extraordinary stars
 And the best part? You'll never miss the animal products, only the culinary adventure


So, whether you're a seasoned vegan or just dipping your toes into the plant-based pool, fire away with your questions, cravings, and cultural preferences
 I'm here to navigate you through the vibrant world of ve

### Prompt follow ups

In [29]:
q = "How do I make butter chicken vegan?"
answer = bard.get_answer(q)

In [30]:
print(
    "\n".join(answer['content'].split("."))
)

Ah, butter chicken! A beloved Indian dish known for its creamy tomato sauce and tender morsels of chicken
 While the original involves dairy and meat, fear not, vegan friends! We can achieve a remarkably similar experience with plant-based ingenuity
 Here are two popular approaches to vegan butter chicken:

**1
 Tofu Butter Chicken:**

**Ingredients:**

* **Tofu:** Extra firm tofu, cubed and pan-fried for a crispy texture

* **Sauce:** Cashew cream (soaked cashews blended with water), coconut milk, tomato puree, onion, garlic, ginger, garam masala, turmeric, cumin, coriander, cayenne pepper, salt, and a touch of maple syrup for balance


**Method:**

* Pan-fry the tofu until golden and crispy

* Sauté the onion and spices in oil

* Blend the cashew cream and other sauce ingredients into a smooth mixture

* Add the sauce to the pan and simmer until thickened

* Gently fold in the tofu and simmer for a few minutes to infuse the flavors

* Serve over basmati rice, naan, or quinoa, garnish

In [35]:
q = "Can you convert the recipe from this website into a vegan version - https://www.joshuaweissman.com/post/italian-beef. There are no dietary restrictions, and I would love to be as close to the original taste and texture as possible"
answer = bard.get_answer(q)

In [36]:
print(
    "\n".join(answer['content'].split("."))
)

Alright, let's veganize Joshua Weissman's Italian Beef! While achieving the exact texture of juicy, shredded beef might be tricky, we can definitely get close with some clever plant-based substitutes and flavor hacks
 Here's your vegan Italian Beef adventure:

**Protein Options:**

1
 **Seitan:** This wheat gluten meat substitute can be shredded and cooked to mimic the texture of pulled beef
 Marinate it in Italian herbs and spices before braising to get that signature Italian Beef flavor

2
 **Oyster mushrooms:** These meaty mushrooms hold their shape well and absorb marinades beautifully
 Pull them apart into strips for a slightly chewier texture

3
 **Jackfruit:** Young green jackfruit has a neutral flavor and a fibrous texture similar to pulled pork
 Cook it in the braising liquid to infuse it with savory Italian flavors

4
 **Lentil and mushroom mixture:** Combine cooked lentils with chopped, sauteed mushrooms and season with Italian herbs and spices
 This option is packed with pr